In [1]:
import os
os.chdir('/home/hugo-dev/esirem-dev/4a/kmutt/kmutt_v2')

In [2]:
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from utils.load_data import load_data
from utils.plot_history import plot_history

2024-05-15 18:02:09.570673: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 18:02:17.906439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
DATA_PATH = "./data/data_25ms_h=50%_16mfcc_RMT-HRK.json"
SAVED_MODEL_PATH = "./models/svm/model.keras"
EPOCHS = 50
BATCH_SIZE = 32
PATIENCE = 5
LEARNING_RATE = 0.0001

In [4]:
def prepare_dataset(data_path, test_size=0.2, validation_size=0.2):
    # load dataset
    X, y = load_data(data_path)

    # initialisez le scaler
    scaler = MinMaxScaler()

    # redimensionnez X en 2D pour le scaling
    X_2d = X.reshape(-1, X.shape[-1])

    # ajustez et transformez les données
    X_scaled = scaler.fit_transform(X_2d)

    # redimensionnez X à sa forme originale
    X_normalized = X_scaled.reshape(X.shape)

    # create train, validation, test split
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, y_train, X_validation, y_validation, X_test, y_test

In [5]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# generate train, validation and test sets
X_train, y_train, X_validation, y_validation, X_test, y_test = prepare_dataset(DATA_PATH)

# flatten the data for SVM
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_validation_flatten = X_validation.reshape(X_validation.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

# create SVM model
clf = svm.SVC(kernel='linear')  # Vous pouvez changer le noyau ici

# train SVM
clf.fit(X_train_flatten, y_train)

# validate the model
validation_predictions = clf.predict(X_validation_flatten)
print("Validation Accuracy: ", accuracy_score(y_validation, validation_predictions))

# evaluate network on test set
test_predictions = clf.predict(X_test_flatten)
test_acc = accuracy_score(y_test, test_predictions)
print("\nTest accuracy: {}".format(100*test_acc))


Training sets loaded!
Validation Accuracy:  0.8023319838056681

Test accuracy: 80.27362475060245
